# WINE ANALYTICS MP2

In [2]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import os
from pathlib import Path
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, QuantileTransformer
from sklearn.feature_selection import SelectKBest, f_classif

In [3]:
# Setup
DATA_PATH = Path("../data")
redWine = DATA_PATH / "winequality-red.xlsx"
whiteWine = DATA_PATH / "winequality-white.xlsx"

# Dataframes for files - header=1 to skip the first row (Title of file)
df_red_original = pd.read_excel(redWine, header=1)
df_white_original = pd.read_excel(whiteWine, header=1)

## Data Cleaning

In [4]:
# Shape of files
print(df_red_original.shape)
print(df_white_original.shape)

(1599, 12)
(4898, 12)


### Add wine type column to dataframe

In [5]:
# Assigning original df's to workable ones
df_red = df_red_original.copy()
df_white = df_white_original.copy()

In [6]:
df_red['wine_type'] = 'red'
df_white['wine_type'] = 'white'


In [7]:
# Ensure new columns has been added
df_red.shape

(1599, 13)

### Finding NA data (if any) and removing duplicate data before anylysis 

In [8]:
print(df_red.isna().sum())
print(df_white.isna().sum())

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
wine_type               0
dtype: int64
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
wine_type               0
dtype: int64


Since no NA data exists, we move on to dropping duplicate data from the dataframes:

In [9]:
# Finding sum of duplicates
print(df_red.duplicated().sum())
print(df_white.duplicated().sum())


240
937


In [10]:
# Dropping duplicates from df's and assigning them to new ones
df_red_clean = df_red.drop_duplicates()
df_white_clean = df_white.drop_duplicates()

# Sum of duplicates after cleaning (should be 0)
print("Sum of red duplicates: " + str(df_red_clean.duplicated().sum()) + "\nNew df_red shape: " + str(df_red_clean.shape))
print("\nSum of white duplicates: "  + str(df_white_clean.duplicated().sum()) + "\nNew df_white shape: " + str(df_white_clean.shape))

Sum of red duplicates: 0
New df_red shape: (1359, 13)

Sum of white duplicates: 0
New df_white shape: (3961, 13)


### Combining the two dataframes into one for processing

In [11]:
df_combined = pd.concat([df_red_clean, df_white_clean], ignore_index=True)

df_combined.shape

(5320, 13)

In [12]:
# Making use both white and red (wine_type) exists in the df
df_combined.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine_type
2165,7.3,0.240,0.34,15.40,0.050,38.0,174.0,0.99830,3.03,0.42,9.000000,6,white
5091,6.1,0.190,0.37,2.60,0.041,24.0,99.0,0.99153,3.18,0.50,10.900000,6,white
2154,8.2,0.345,1.00,18.20,0.047,55.0,205.0,0.99965,2.96,0.43,9.600000,5,white
1489,7.2,0.180,0.41,1.20,0.048,41.0,97.0,0.99190,3.14,0.45,10.400000,5,white
2582,8.3,0.200,0.74,4.45,0.044,33.0,130.0,0.99240,3.25,0.42,12.200000,6,white
3877,7.4,0.210,0.80,12.30,0.038,77.0,183.0,0.99778,2.95,0.48,9.000000,5,white
1155,8.0,0.830,0.27,2.00,0.080,11.0,63.0,0.99652,3.29,0.48,9.800000,4,red
2422,6.4,0.170,0.27,6.70,0.036,88.0,223.0,0.99480,3.28,0.35,10.200000,6,white
4901,5.2,0.220,0.46,6.20,0.066,41.0,187.0,0.99362,3.19,0.42,9.733333,5,white
3952,7.8,0.300,0.36,4.60,0.024,20.0,198.0,0.99222,3.06,0.66,11.900000,6,white


## Data Exploring 

### Exploring the features of the three df's separately

In [13]:
# Red wine
df_red_original.sample()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
172,8.0,0.42,0.17,2.0,0.073,6.0,18.0,0.9972,3.29,0.61,9.2,6


In [14]:
# White wine
df_white_original.sample()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1202,6.4,0.3,0.38,7.8,0.046,35.0,192.0,0.9955,3.1,0.37,9.0,5


In [15]:
df_combined.sample()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine_type
2679,7.6,0.18,0.49,18.05,0.046,36.0,158.0,0.9996,3.06,0.41,9.2,5,white


The original dataframes for the red and white wine have a single dependent variable 'quality', where the others are independent (at least from the initial assumption). 
The combined dataframe potentially has two dependent variables after we've added the 'wine_type' variable.

## Exploring features

In [16]:
df_red_clean.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000
mean,8.310596,0.529478,0.272333,2.523400,0.088124,15.893304,46.825975,0.996709,3.309787,0.658705,10.432315,5.623252
std,1.736990,0.183031,0.195537,1.352314,0.049377,10.447270,33.408946,0.001869,0.155036,0.170667,1.082065,0.823578
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996700,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.430000,2.600000,0.091000,21.000000,63.000000,0.997820,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [17]:
df_white_clean.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,3961.000000,3961.000000,3961.000000,3961.000000,3961.000000,3961.000000,3961.000000,3961.000000,3961.000000,3961.000000,3961.000000,3961.000000
mean,6.839346,0.280538,0.334332,5.914819,0.045905,34.889169,137.193512,0.993790,3.195458,0.490351,10.589358,5.854835
std,0.866860,0.103437,0.122446,4.861646,0.023103,17.210021,43.129065,0.002905,0.151546,0.113523,1.217076,0.890683
min,3.800000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.300000,0.210000,0.270000,1.600000,0.035000,23.000000,106.000000,0.991620,3.090000,0.410000,9.500000,5.000000
50%,6.800000,0.260000,0.320000,4.700000,0.042000,33.000000,133.000000,0.993500,3.180000,0.480000,10.400000,6.000000
75%,7.300000,0.330000,0.390000,8.900000,0.050000,45.000000,166.000000,0.995710,3.290000,0.550000,11.400000,6.000000
max,14.200000,1.100000,1.660000,65.800000,0.346000,289.000000,440.000000,1.038980,3.820000,1.080000,14.200000,9.000000


In [18]:
df_combined.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,5320.000000,5320.000000,5320.000000,5320.000000,5320.000000,5320.000000,5320.000000,5320.000000,5320.000000,5320.000000,5320.000000,5320.000000
mean,7.215179,0.344130,0.318494,5.048477,0.056690,30.036654,114.109023,0.994535,3.224664,0.533357,10.549241,5.795677
std,1.319671,0.168248,0.147157,4.500180,0.036863,17.805045,56.774223,0.002966,0.160379,0.149743,1.185933,0.879772
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.400000,0.230000,0.240000,1.800000,0.038000,16.000000,74.000000,0.992200,3.110000,0.430000,9.500000,5.000000
50%,7.000000,0.300000,0.310000,2.700000,0.047000,28.000000,116.000000,0.994650,3.210000,0.510000,10.400000,6.000000
75%,7.700000,0.410000,0.400000,7.500000,0.066000,41.000000,153.250000,0.996770,3.330000,0.600000,11.400000,6.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000


### Transforming categorical data (wine_type)

In [19]:
df_combined['wine_type'] = df_combined['wine_type'].astype('category').cat.codes

In [20]:
df_combined.sample(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine_type
4167,7.8,0.290,0.29,3.15,0.044,41.0,117.0,0.99153,3.24,0.35,11.5,5,1
1407,6.9,0.250,0.35,1.30,0.039,29.0,191.0,0.99080,3.13,0.52,11.0,6,1
1551,6.4,0.170,0.32,2.40,0.048,41.0,200.0,0.99380,3.50,0.50,9.7,6,1
1165,6.8,0.815,0.00,1.20,0.267,16.0,29.0,0.99471,3.32,0.51,9.8,3,0
2855,7.7,0.430,0.28,4.50,0.046,33.0,102.0,0.99180,3.16,0.56,12.2,7,1


In [21]:
categorical_data = df_combined.drop(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality'], axis=1)

In [22]:
wine_type_count = categorical_data['wine_type'].value_counts()

In [23]:
wine_type_count

wine_type
1    3961
0    1359
Name: count, dtype: int64

## Visualization